In [ ]:
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table, join, vstack
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.io import ascii

import gala.coordinates as gc
import gala.dynamics as gd
from gala.units import galactic
from pyia import GaiaData

In [ ]:
pm_poly = np.load('../../gd1-dr2/output/pm_poly.npy')

In [ ]:
w = np.load('../../gd1-dr2/data/stream_model.npy')
stream_w = gd.PhaseSpacePosition(pos=w[:, :3].T*u.kpc,
                                 vel=w[:, 3:].T*u.km/u.s)
model_c = stream_w.to_coord_frame(gc.GD1)
model_c = model_c[(model_c.phi1.wrap_at(180*u.deg) < 20*u.deg) & 
                  (model_c.phi1.wrap_at(180*u.deg) > -100*u.deg)]
model_c_ref = gc.reflex_correct(model_c)

In [ ]:
gd1_g = GaiaData('../../gd1-dr2/data/gd1-master.fits')

In [ ]:
xmatch = Table.read('/Users/adrian/data/APOGEE_DR15beta/gaiadr2_allStar-t9-l31c-58158.fits')
xmatch.rename_column('apstar_id', 'APSTAR_ID')

In [ ]:
apogee = Table.read('/Users/adrian/data/APOGEE_DR15beta/allStar-t9-l31c-58158.fits')

In [ ]:
stream_mask = (apogee['APOGEE2_TARGET1'] & (2**18 + 2**19)) != 0
stream_mask.sum()

In [ ]:
apogee_gaia = join(apogee[stream_mask], xmatch, keys='APSTAR_ID')
ndim_cols = [x for x in apogee_gaia.columns if apogee_gaia[x].ndim == 1]
apogee_gaia = apogee_gaia[ndim_cols]

# select distance stuff
apogee_gaia = apogee_gaia[(apogee_gaia['parallax'] < 1.) | ((apogee_gaia['parallax'] - apogee_gaia['parallax_error']) < 1.)]

g = GaiaData(apogee_gaia)
len(g)

In [ ]:
mask = gd1_mask

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

ax = axes[0]
ax.plot(apogee_gaia['TEFF'][mask], apogee_gaia['LOGG'][mask], 
        marker='.', linestyle='none')
ax.set_xlabel('Teff')

ax.set_xlim(6500, 3500)
ax.set_ylim(4, 0)
ax.set_ylabel('logg')

ax = axes[1]
ax.plot(apogee_gaia['RA'][mask], apogee_gaia['DEC'][mask], 
        marker='.', linestyle='none')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
ax.set_xlim(360, 0)
ax.set_ylim(-30, 90)

fig.tight_layout()

In [ ]:
gd1_mask = np.array(['GD1' in f for f in apogee_gaia['FIELD']])
gd1_mask.sum()

In [ ]:
c = g.get_skycoord(distance=8*u.kpc, 
                   radial_velocity=np.array(g.VHELIO_AVG)*u.km/u.s)
c = gc.reflex_correct(c.transform_to(gc.GD1))

phi1 = c.phi1.wrap_at(180*u.deg)
phi2 = c.phi2

pm1 = c.pm_phi1_cosphi2.to(u.mas/u.yr)
pm2 = c.pm_phi2.to(u.mas/u.yr)

In [ ]:
gd1_c = gd1_g[gd1_g.gi_cmd_mask & gd1_g.pm_mask].get_skycoord(distance=False)
gd1_c = gd1_c.transform_to(gc.GD1)

gd1_phi1 = gd1_c.phi1.wrap_at(180*u.deg)
gd1_phi2 = gd1_c.phi2
gd1_pm1 = gd1_c.pm_phi1_cosphi2.to(u.mas/u.yr)
gd1_pm2 = gd1_c.pm_phi2.to(u.mas/u.yr)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 3))

ax.plot(phi1[gd1_mask], phi2[gd1_mask], 
        marker='o', mec='tab:blue', mew=1, mfc='none',
        ls='none', alpha=0.5)

ax.plot(gd1_phi1, gd1_phi2, 
        marker='.', ls='none', alpha=0.5, color='k')

ax.set_xlim(-100, 20)
ax.set_ylim(-10, 5)
ax.set_aspect('equal')

ax.set_xlabel(r'$\phi_1$')
ax.set_ylabel(r'$\phi_2$')

fig.tight_layout()
# fig.savefig('../plots/full-phi1phi2.png', dpi=250)

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for i, field_name in enumerate(np.unique(apogee_gaia['FIELD'][gd1_mask])):
    ax = axes[i]
    
    field_mask = apogee_gaia['FIELD'] == field_name

    _phi1 = phi1[gd1_mask & field_mask].value
    _phi2 = phi2[gd1_mask & field_mask].value
    
    ax.scatter(phi1[gd1_mask], phi2[gd1_mask], 
               marker='o', color='tab:blue', linewidth=1, facecolor='none',
               alpha=1, s=25)

    ax.plot(gd1_phi1, gd1_phi2, 
            marker='o', ls='none', alpha=0.75, color='k')

    ax.set_xlim(np.mean(_phi1) - 3.5, 
                np.mean(_phi1) + 3.5)
    ax.set_ylim(np.mean(_phi2) - 3.5, 
                np.mean(_phi2) + 3.5)
    ax.set_aspect('equal')
    
    ax.set_xlabel(r'$\phi_1$ [deg]')

    fig.tight_layout()
    
axes[0].set_ylabel(r'$\phi_2$ [deg]')

fig.savefig('../plots/zoom-phi1phi2.png', dpi=200)

In [ ]:
kop_vr = ascii.read("""phi1 phi2 vr err
-45.23 -0.04 28.8 6.9
-43.17 -0.09 29.3 10.2
-39.54 -0.07 2.9  8.7
-39.25 -0.22 -5.2 6.5
-37.95 0.00 1.1   5.6
-37.96 -0.00 -11.7 11.2
-35.49 -0.05 -50.4 5.2
-35.27 -0.02 -30.9 12.8
-34.92 -0.15 -35.3 7.5
-34.74 -0.08 -30.9 9.2
-33.74 -0.18 -74.3 9.8
-32.90 -0.15 -71.5 9.6
-32.25 -0.17 -71.5 9.2
-29.95 -0.00 -92.7 8.7
-26.61 -0.11 -114.2 7.3
-25.45 -0.14 -67.8 7.1
-24.86 0.01 -111.2 17.8
-21.21 -0.02 -144.4 10.5
-14.47 -0.15 -179.0 10.0
-13.73 -0.28 -191.4 7.5
-13.02 -0.21 -162.9 9.6
-12.68 -0.26 -217.2 10.7
-12.55 -0.23 -172.2 6.6""")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.scatter(pm1[gd1_mask], pm2[gd1_mask], marker='o', color='tab:blue', 
           linewidth=1, s=15, alpha=1)

pm_path = mpl.patches.Polygon(pm_poly, 
                              facecolor='none', edgecolor='k')
ax.add_patch(pm_path)
pm_path = mpl.patches.Path(pm_poly)

ax.scatter(model_c_ref.pm_phi1_cosphi2.to(u.mas/u.yr), 
           model_c_ref.pm_phi2.to(u.mas/u.yr), 
           zorder=-100, color='#aaaaaa')

ax.set_xlim(-15, 15)
ax.set_ylim(-15, 15)

ax.set_xlabel(r'$\mu_1 \, \cos\phi_2$')
ax.set_ylabel(r'$\mu_2$')
fig.tight_layout()
fig.savefig('../plots/1-pm.png', dpi=250)

In [ ]:
apogee_pm_mask = pm_path.contains_points(np.vstack((pm1, pm2)).T)
apogee_pm_mask.sum()

In [ ]:
(gd1_mask & apogee_pm_mask).sum()

In [ ]:
g.M_H[gd1_mask & apogee_pm_mask]

In [ ]:
members_mask = gd1_mask & apogee_pm_mask & (g.FE_H < -1) & (g.FE_H > -99)
elems = np.array(['M_H', 'ALPHA_M', 'MG_FE', 'CA_FE', 'SI_FE', 'N_FE', 
                 'MN_FE', 'Y_FE', 'V_FE', 'C_FE', 'CO_FE', 'CR_FE', 
                  'S_FE', 'AL_FE'])

elems1 = np.array([g.data[members_mask][e][0] for e in elems])
elems2 = np.array([g.data[members_mask][e][1] for e in elems])
diffs = elems1 - elems2
errs = np.sqrt(np.array([g.data[members_mask][e+'_ERR'][0] for e in elems])**2 + 
               np.array([g.data[members_mask][e+'_ERR'][1] for e in elems])**2)

good_mask = (elems1 > -99) & (elems2 > -99)

fig, ax = plt.subplots(1, 1, figsize=(10, 5))

ax.errorbar(elems[good_mask], diffs[good_mask],
            yerr=errs[good_mask], marker='o', ls='none')

ax.axhline(0, zorder=-10, marker='', alpha=0.2)
ax.set_ylim(-1, 1)

plt.xticks(rotation='vertical')

ax.set_ylabel(r'$\Delta$')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

_phi1 = phi1[gd1_mask].value
_rv = g.VHELIO_AVG[gd1_mask]

ax.scatter(_phi1, _rv, 
           marker='o', color='tab:blue', linewidth=1, facecolor='none',
           alpha=1, s=25, label='APOGEE')

# ax.scatter(_phi1, g.radial_velocity[gd1_mask], 
#            color='tab:purple')

ax.scatter(phi1.value[members_mask],
           g.VHELIO_AVG[members_mask], 
           color='tab:red', label='[Fe/H]+Gaia members')

ax.plot(kop_vr['phi1'], kop_vr['vr'], 
        marker='o', ls='none', alpha=0.75, color='k', 
        label='Koposov/SEGUE')

ax.scatter(model_c.phi1.wrap_at(180*u.deg),
           model_c.radial_velocity.to(u.km/u.s), 
           zorder=-100, color='#aaaaaa',
           label='model')

ax.set_xlim(-100, 20)
ax.set_ylim(-350, 350)

ax.set_xlabel(r'$\phi_1$ [deg]')

ax.legend(loc='upper right', fontsize=18)
ax.set_ylabel(r'$v_r$ [{0:latex_inline}]'.format(u.km/u.s))
fig.tight_layout()
fig.savefig('../plots/1-rv.png', dpi=250)

### Try RV selection instead:

In [ ]:
wtf_mask = (((phi1.value < -50) & (phi1.value > -60) & (rv > 93) & (rv < 150)) | 
            ((phi1.value < -20) & (phi1.value > -30) & (rv < -80) & (rv > -200)) |
            ((phi1.value < 20) & (phi1.value > 0) & (rv < -220) & (rv > -300)))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

_phi1 = phi1[gd1_mask].value
_rv = g.VHELIO_AVG[gd1_mask]

ax.scatter(_phi1, _rv, 
           marker='o', color='tab:blue', linewidth=1, facecolor='none',
           alpha=1, s=25, label='APOGEE')

# ax.scatter(_phi1, g.radial_velocity[gd1_mask], 
#            color='tab:purple')

ax.scatter(phi1.value[gd1_mask & wtf_mask],
           g.VHELIO_AVG[gd1_mask & wtf_mask], 
           color='tab:red', label='RV members')

ax.plot(kop_vr['phi1'], kop_vr['vr'], 
        marker='o', ls='none', alpha=0.75, color='k', 
        label='Koposov/SEGUE')

ax.scatter(model_c.phi1.wrap_at(180*u.deg),
           model_c.radial_velocity.to(u.km/u.s), 
           zorder=-100, color='#aaaaaa',
           label='model')

ax.set_xlim(-100, 20)
ax.set_ylim(-350, 350)

ax.set_xlabel(r'$\phi_1$ [deg]')

ax.legend(loc='upper right', fontsize=18)
ax.set_ylabel(r'$v_r$ [{0:latex_inline}]'.format(u.km/u.s))
fig.tight_layout()
fig.savefig('../plots/2-rv.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.scatter(pm1[gd1_mask], pm2[gd1_mask], marker='o', color='tab:blue', 
           linewidth=1, s=15, alpha=1)

ax.scatter(pm1[gd1_mask & wtf_mask], pm2[gd1_mask & wtf_mask], marker='o', color='tab:red', 
           linewidth=1, s=15, alpha=1)

pm_path = mpl.patches.Polygon(pm_poly, 
                              facecolor='none', edgecolor='k')
ax.add_patch(pm_path)
pm_path = mpl.patches.Path(pm_poly)

ax.scatter(model_c_ref.pm_phi1_cosphi2.to(u.mas/u.yr), 
           model_c_ref.pm_phi2.to(u.mas/u.yr), 
           zorder=-100, color='#aaaaaa')

ax.set_xlim(-15, 15)
ax.set_ylim(-15, 15)

ax.set_xlabel(r'$\mu_1 \, \cos\phi_2$')
ax.set_ylabel(r'$\mu_2$')

fig.savefig('../plots/2-pm.png', dpi=250)
fig.set_facecolor('w')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

_phi1 = phi1[gd1_mask].value
_mh = g.M_H[gd1_mask]

ax.scatter(_phi1, _mh, 
           marker='o', color='tab:blue', linewidth=1, facecolor='none',
           alpha=1, s=25, label='APOGEE')

ax.scatter(phi1.value[gd1_mask & wtf_mask],
           g.M_H[gd1_mask & wtf_mask], 
           color='tab:red', label='RV members')

ax.set_xlim(-100, 20)
ax.set_ylim(-2.5, 0.75)

ax.set_xlabel(r'$\phi_1$ [deg]')

ax.legend(loc='upper right', fontsize=18)
ax.set_ylabel(r'[Fe/H]')
fig.tight_layout()
fig.savefig('../plots/2-feh.png', dpi=250)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 8), 
                         sharex=True, sharey=True)

ax = axes[0]
ax.scatter(phi1[gd1_mask], pm1[gd1_mask],
           c='k', marker='o', alpha=0.5, linewidth=0)

ax.scatter(model_c.phi1.wrap_at(180*u.deg),
           model_c.pm_phi1_cosphi2.to(u.mas/u.yr), 
           zorder=-100, color='#aaaaaa')

ax.set_ylabel('pm1')

ax.set_xlim(-100, 30)
ax.set_ylim(-20, 15)


ax = axes[1]
ax.scatter(phi1[gd1_mask], pm2[gd1_mask],
           c='k', marker='o', alpha=0.5, linewidth=0)

ax.scatter(model_c.phi1.wrap_at(180*u.deg),
           model_c.pm_phi2.to(u.mas/u.yr), 
           zorder=-100, color='#aaaaaa')

ax.set_ylabel('pm2')
ax.set_xlabel('phi1')

fig.tight_layout()